In [16]:
import boto3

# ✅ Crear el cliente de SageMaker
sagemaker_client = boto3.client("sagemaker")

# 🔄 Ahora puedes listar los endpoints
endpoints_list = sagemaker_client.list_endpoints()
print("📌 Lista de endpoints disponibles:", [ep["EndpointName"] for ep in endpoints_list["Endpoints"]])

📌 Lista de endpoints disponibles: ['sagemaker-scikit-learn-2025-05-07-06-04-40-195', 'sagemaker-scikit-learn-2025-05-07-04-45-43-055']


In [17]:
endpoint_name = "nombre_real_de_tu_endpoint"  # 🔥 Ajusta con el nombre correcto

import boto3
sagemaker_client = boto3.client("sagemaker")
response = sagemaker_client.describe_endpoint(EndpointName='sagemaker-scikit-learn-2025-05-07-06-04-40-195')

print("🟢 Estado del endpoint:", response["EndpointStatus"])

🟢 Estado del endpoint: InService


In [33]:
import boto3
import json
import pandas as pd
import os

from transform_functions import fun_final_base, NonNullValues, new_vars_trend, new_vars_trend, encoder_scaler_loading, const_vars_serv, transf_PCA_POL

# Cliente de SageMaker Runtime
sagemaker_runtime = boto3.client("sagemaker-runtime")

# Nombre del endpoint
endpoint_name = 'sagemaker-scikit-learn-2025-05-07-06-04-40-195'  # Ajusta según tu configuración

# Leer el CSV y convertirlo en JSON
df = pd.read_csv("df_muestra.csv")  # Reemplaza con la ruta correcta
payload = df.to_json(orient="records")  # Convierte los datos en formato JSON

# Enviar la solicitud al modelo
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=payload
)

# Mostrar resultado
result = json.loads(response["Body"].read().decode("utf-8"))
print("🔮 Predicciones:", result)

🔮 Predicciones: [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [38]:
import boto3
import json
import pandas as pd
import os

from transform_functions import fun_final_base, NonNullValues, new_vars_trend, new_vars_trend, encoder_scaler_loading, const_vars_serv, transf_PCA_POL

# Cliente de SageMaker Runtime
sagemaker_runtime = boto3.client("sagemaker-runtime")

# Nombre del endpoint
endpoint_name = 'sagemaker-scikit-learn-2025-05-07-06-04-40-195'  # Ajusta según tu configuración

# Leer el CSV y convertirlo en JSON
df = pd.read_csv("df_muestra.csv")

# ✅ Seleccionar solo la primera fila
sample_payload = json.dumps([df.to_dict(orient="records")[0]])  # 🔥 Primera fila

# Enviar la solicitud al modelo
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=sample_payload
)

# Mostrar resultado
result = json.loads(response["Body"].read().decode("utf-8"))
print("🔮 Predicciones:", result)

🔮 Predicciones: [1]


In [68]:
import json
import pandas as pd

url = "https://dtpnqzhts3.execute-api.us-east-2.amazonaws.com/Prueba"

# 📂 Leer el CSV y convertirlo en JSON
df = pd.read_csv("df_muestra.csv")

# 🚨 Manejo de NaN: Reemplazar valores `NaN` con `None` para JSON válido
df = df.where(pd.notna(df), None)

# ✅ Seleccionar solo la primera fila con la clave esperada en Lambda
sample_payload = {"inputs": [df.to_dict(orient="records")[0]]}  # ✅ Estructura esperada

# ✅ Envolver en formato esperado por Lambda
lambda_event = {"body": json.dumps(sample_payload)}  # 🔥 Convertir `sample_payload` a string JSON dentro de `body`

# 🚀 Enviar la solicitud
response = requests.post(url, json=lambda_event)
print(response.json())  # 🔥 Revisar la respuesta

{'statusCode': 200, 'body': '{"predictions": [1]}'}


In [74]:
import requests
import pandas as pd
import json

url = "https://dtpnqzhts3.execute-api.us-east-2.amazonaws.com/Prueba"

# 📂 Leer el CSV y convertirlo en JSON
df = pd.read_csv("df_muestra.csv")

# 🚨 Manejo de NaN: Reemplazar valores `NaN` con `None` para JSON válido
df = df.where(pd.notna(df), None)

# ✅ Lista para almacenar los resultados
results = []

# 🔄 Iterar sobre cada fila del DataFrame
for i in range(df.shape[0]):  
    # 📌 Obtener ID de la fila actual
    ID = df.iloc[i]["ID"]

    # ✅ Seleccionar la fila `i` y convertirla a JSON
    sample_payload = {"inputs": [df.iloc[i].to_dict()]}  
    
    # ✅ Envolver en formato esperado por Lambda
    lambda_event = {"body": json.dumps(sample_payload)}  
    
    # 🚀 Enviar la solicitud
    response = requests.post(url, json=lambda_event)
    
    # 📌 Extraer `prediction` desde la respuesta
    prediction = json.loads(response.json()["body"])["predictions"][0]  
    
    # 📌 Almacenar en la lista
    results.append({"ID": ID, "prediction": prediction})  

# ✅ Crear DataFrame con los resultados
df_results = pd.DataFrame(results)

# 🔍 Verificación
print(df_results.head())  # 🔥 Revisa la estructura del DataFrame

       ID  prediction
0   90659           1
1  473019           1
2  421306           1
3  291700           1
4  447516           1


In [75]:
df_results

,ID,prediction
0,90659,1
1,473019,1
2,421306,1
3,291700,1
4,447516,1
...,...,...
95,458401,0
96,132432,0
97,165854,0
98,237372,0
